# 🧠 Soil Classification - Training Notebook
Welcome to the training notebook for our soil classification challenge. In this notebook, we:
- Load labeled soil + synthetic data
- Train a classifier to detect "soil" vs "not soil"
- Track F1 score
- Save the best model

---


We'll split this into cells as they would appear in a notebook

## Cell 1: Imports & Setup

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm

In [ ]:
### 🖼️ Step 2: Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


 Cell 3: Transforms and Datasets

In [ ]:
train_transform, val_transform = get_transforms()

train_dataset = SoilDataset(train_df, TRAIN_DIR, train_transform)
val_dataset = SoilDataset(val_df, TRAIN_DIR, val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)


✅ Using device: cuda (NVIDIA GeForce RTX 4050 Laptop GPU)


c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


 cuda setup and check , working on laptop vs code locally

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1),
    nn.Sigmoid()
)
model = model.to(device)


PyTorch version: 2.5.1+cu121
CUDA available: True
GPU device name: NVIDIA GeForce RTX 4050 Laptop GPU


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1),
    nn.Sigmoid()
)
model = model.to(device)


CUDA Available: True
GPU Device: NVIDIA GeForce RTX 4050 Laptop GPU
Output shape: torch.Size([8, 1000])


In [ ]:
### 📂 Step 4: Paths and CSVs
train_csv = '/kaggle/input/soil-classification-part-2/soil_competition-2025/train_labels.csv'
test_csv = '/kaggle/input/soil-classification-part-2/soil_competition-2025/test_ids.csv'
train_dir = '/kaggle/input/soil-classification-part-2/soil_competition-2025/train'
test_dir = '/kaggle/input/soil-classification-part-2/soil_competition-2025/test'

train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)


Loss & Optimizer

In [ ]:
### 🧺 Step 5: Custom Dataset
class SoilDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform):
        self.df = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_id = self.df.iloc[idx]['image_id']
        image_path = os.path.join(self.img_dir, image_id)
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)
        return image, image_id

Training Function

In [ ]:
def train_model(model, train_loader, val_loader, epochs=10):
    best_f1 = 0
    best_threshold = 0.5

    for epoch in range(epochs):
        model.train()
        train_loss = 0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation
        model.eval()
        all_probs = []
        all_targets = []

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                all_probs.extend(outputs.cpu().numpy().flatten())
                all_targets.extend(labels.cpu().numpy())

        all_probs = np.array(all_probs)
        all_targets = np.array(all_targets)

        # Threshold tuning
        thresholds = np.arange(0.1, 0.9, 0.01)
        f1_scores = []

        for t in thresholds:
            preds = (all_probs > t).astype(int)
            f1 = f1_score(all_targets, preds)
            f1_scores.append(f1)

        max_f1 = max(f1_scores)
        max_thresh = thresholds[np.argmax(f1_scores)]

        if max_f1 > best_f1:
            best_f1 = max_f1
            best_threshold = max_thresh
            torch.save(model.state_dict(), './model_best.pth')  # Save best model

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val F1: {max_f1:.4f} at threshold {max_thresh:.2f}")

    return best_threshold


In [ ]:
train_loader = DataLoader(SoilDataset(train_df, train_dir, transform), batch_size=32, shuffle=False)
test_loader = DataLoader(SoilDataset(test_df, test_dir, transform), batch_size=32, shuffle=False)


In [ ]:
### 🔍 Step 6: Feature Extraction
def extract_features(dataloader, model, device):
    features = []
    ids = []
    with torch.no_grad():
        for images, image_ids in tqdm(dataloader, desc="Extracting features"):
            images = images.to(device)
            feats = model(images).cpu().numpy()
            features.append(feats)
            ids.extend(image_ids)
    return np.vstack(features), ids

train_features, _ = extract_features(train_loader, resnet, device)
test_features, test_ids = extract_features(test_loader, resnet, device)

In [1]:
pip install torchviz


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 💾 Step 7: Save Features

In [ ]:

np.save("train_features.npy", train_features)
np.save("test_features.npy", test_features)
np.save("test_ids.npy", test_ids)

